In [1]:
%matplotlib inline

In [2]:
import os
import torch

In [3]:
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
print(device)

cuda:7


In [4]:
%load_ext autoreload
%autoreload 2

# load data

In [5]:
from transformer_translation.dataset import ParallelLanguageDataset
from torch.utils.data import DataLoader

In [6]:
data_path = r"/home/alex/data/nlp/agmir/transf_processed_data"
#data_path = 'transformer_translation/data/processed'

In [66]:
num_tokens = 2000
max_seq_length = 96
dataset = ParallelLanguageDataset(
    os.path.join(data_path, 'tags/set.pkl')#'en/train.pkl')#
    ,os.path.join(data_path, 'reports/set.pkl')#'fr/train.pkl')#
    ,num_tokens
    ,max_seq_length)
loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)

In [67]:
tk1 = [j for i in dataset.data_1 for j in i]
tk2 = [j for i in dataset.data_2 for j in i]

In [68]:
(max(tk1)), (max(tk2))

(589, 1952)

In [26]:
dataset.data_lengths

{(2, 58): [1819, 2725, 140, 1223, 983, 0, 621],
 (9, 49): [2],
 (1, 45): [3,
  546,
  257,
  2253,
  2980,
  3152,
  973,
  1616,
  2533,
  3358,
  1573,
  3289,
  802,
  3165,
  1713,
  1112,
  1340,
  2936,
  1678,
  1804,
  3286,
  1416,
  735,
  851,
  2913,
  2914,
  2772,
  2199,
  1885,
  1483,
  2238,
  1305,
  1758,
  2655,
  1847],
 (2, 39): [2933, 3229, 1439, 5, 335, 176],
 (1, 21): [595,
  92,
  330,
  2785,
  1333,
  107,
  1215,
  2246,
  2088,
  2374,
  3539,
  2797,
  631,
  3044,
  1167,
  2125,
  1491,
  3178,
  2316,
  266,
  2928,
  2298,
  3383,
  345,
  1239,
  2498,
  2510,
  2118,
  1647,
  6,
  829,
  2338,
  2643,
  175,
  346,
  2671,
  2016,
  40,
  3010,
  1234,
  2391,
  1708,
  2844,
  3063,
  3502,
  3503,
  729,
  3369,
  74,
  2119,
  336,
  2411,
  1821,
  925,
  2650,
  3382,
  2239,
  2753,
  3228,
  2208,
  820,
  3134,
  2504,
  875,
  2562,
  3090,
  1193,
  970,
  91,
  3148,
  2217,
  2929,
  1826],
 (1, 50): [2314,
  2623,
  255,
  3559,
  341

# train

In [78]:
from transformer_translation.model import LanguageTransformer
from transformer_translation.translate_sentence import gen_nopeek_mask

In [81]:
vocab_size = 10000 + 4#1952#
d_model = 512
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048
pos_dropout = 0.1
trans_dropout = 0.1

model = LanguageTransformer(
    vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward,
    max_seq_length, pos_dropout, trans_dropout
).to(device)

In [82]:
    from transformer_translation.Optim import ScheduledOptim
    import torch.nn as nn
    from torch.optim import Adam
    
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_normal_(p)

    n_warmup_steps = 4000
    optim = ScheduledOptim(
        Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09),
        d_model, n_warmup_steps)

    criterion = nn.CrossEntropyLoss(ignore_index=0)

In [83]:
(src, src_key_padding_mask, tgt, tgt_key_padding_mask) = next(iter(loader))

In [84]:
src.shape, src_key_padding_mask.shape

(torch.Size([1, 62, 1]), torch.Size([1, 62, 1]))

In [85]:
tgt.shape, tgt_key_padding_mask.shape

(torch.Size([1, 62, 33]), torch.Size([1, 62, 33]))

In [86]:
            src, src_key_padding_mask = src[0].to(device), src_key_padding_mask[0].to(device)
            tgt, tgt_key_padding_mask = tgt[0].to(device), tgt_key_padding_mask[0].to(device)

            memory_key_padding_mask = src_key_padding_mask.clone()
            tgt_inp, tgt_out = tgt[:, :-1], tgt[:, 1:]
            tgt_mask = gen_nopeek_mask(tgt_inp.shape[1]).to(device)

In [76]:
src.shape, model.pos_enc(model.embed_src(rearrange(src, 'n s -> s n')) * math.sqrt(model.d_model)).shape, tgt_inp.shape, src_key_padding_mask.shape, tgt_key_padding_mask[:, :-1].shape, memory_key_padding_mask.shape, tgt_mask.shape

(torch.Size([47, 4]),
 torch.Size([4, 47, 512]),
 torch.Size([47, 95]),
 torch.Size([47, 4]),
 torch.Size([47, 95]),
 torch.Size([47, 4]),
 torch.Size([95, 95]))

In [44]:
import math
tgt.shape, rearrange(tgt, 'n t -> t n').shape, model.pos_enc(model.embed_tgt(rearrange(tgt, 'n t -> t n')) * math.sqrt(model.d_model)).shape

(torch.Size([22, 43]), torch.Size([43, 22]), torch.Size([43, 22, 512]))

In [58]:
src.shape, rearrange(src, 'n s -> s n').shape, model.pos_enc(model.embed_src(rearrange(src, 'n s -> s n')) * math.sqrt(model.d_model)).shape

(torch.Size([31, 2]), torch.Size([2, 31]), torch.Size([2, 31, 512]))

In [87]:
outputs = model(src, tgt_inp, src_key_padding_mask, tgt_key_padding_mask[:, :-1], memory_key_padding_mask, tgt_mask)

torch.float32 torch.bool


In [77]:
src[0].dtype, src.dtype, tgt_inp.dtype, src_key_padding_mask.dtype, tgt_key_padding_mask[:, :-1].dtype, memory_key_padding_mask.dtype, tgt_mask.dtype

(torch.int64,
 torch.int64,
 torch.int64,
 torch.bool,
 torch.bool,
 torch.bool,
 torch.int64)

In [37]:
rearrange(src, 'n s -> s n').shape

torch.Size([2, 38])

In [21]:
    %%time
    print_every = 15
    num_epochs = 10
    model.train()

    lowest_val = 1e9
    val_losses = []
    total_step = 0
    
    for epoch in range(num_epochs):
        
        total_loss = 0

        for step, (src, src_key_padding_mask, tgt, tgt_key_padding_mask) in enumerate(iter(loader)):
            total_step += 1

            src, src_key_padding_mask = src[0].to(device), src_key_padding_mask[0].to(device)
            tgt, tgt_key_padding_mask = tgt[0].to(device), tgt_key_padding_mask[0].to(device)

            memory_key_padding_mask = src_key_padding_mask.clone()
            tgt_inp, tgt_out = tgt[:, :-1], tgt[:, 1:]
            tgt_mask = gen_nopeek_mask(tgt_inp.shape[1]).to(device)

            optim.zero_grad()
            outputs = model(src, tgt_inp, src_key_padding_mask, tgt_key_padding_mask[:, :-1], memory_key_padding_mask, tgt_mask)
            loss = criterion(rearrange(outputs, 'b t v -> (b t) v'), rearrange(tgt_out, 'b o -> (b o)'))

            loss.backward()
            optim.step_and_update_lr()

            total_loss += loss.item()
            if step % print_every == print_every - 1:
                print(f'Epoch [{epoch + 1} / {num_epochs}] \t Step [{step + 1} / {len(loader)}] \t '
                      f'Train Loss: {total_loss / print_every}')
                total_loss = 0
                #print(get_sent_from_tk(
                #    rearrange(outputs, 'b t v -> (b t) v').max(dim=1)[1].unique()
                #    ,reports_index2word))

Epoch [1 / 10] 	 Step [15 / 92] 	 Train Loss: 1.3208194176355998
Epoch [1 / 10] 	 Step [30 / 92] 	 Train Loss: 1.6033766587575278
Epoch [1 / 10] 	 Step [45 / 92] 	 Train Loss: 1.5051896969477336
Epoch [1 / 10] 	 Step [60 / 92] 	 Train Loss: 1.8119399348894756
Epoch [1 / 10] 	 Step [75 / 92] 	 Train Loss: 1.4665009458859761
Epoch [1 / 10] 	 Step [90 / 92] 	 Train Loss: 1.6129396160443623
Epoch [2 / 10] 	 Step [15 / 92] 	 Train Loss: 1.4453423062960307
Epoch [2 / 10] 	 Step [30 / 92] 	 Train Loss: 1.3141321460405986
Epoch [2 / 10] 	 Step [45 / 92] 	 Train Loss: 1.4676307280858358
Epoch [2 / 10] 	 Step [60 / 92] 	 Train Loss: 1.6809895277023315
Epoch [2 / 10] 	 Step [75 / 92] 	 Train Loss: 1.5172077020009358
Epoch [2 / 10] 	 Step [90 / 92] 	 Train Loss: 1.5481433153152466
Epoch [3 / 10] 	 Step [15 / 92] 	 Train Loss: 1.3572155793507894
Epoch [3 / 10] 	 Step [30 / 92] 	 Train Loss: 1.3383694112300872
Epoch [3 / 10] 	 Step [45 / 92] 	 Train Loss: 1.4461939374605814
Epoch [3 / 10] 	 Step [60

# get nat lg

In [16]:
import pickle
with open(os.path.join(data_path, 'tags', 'voc.pkl'), 'rb') as f:
    tags_word2index = pickle.load(f)
tags_index2word = dict(zip(tags_word2index.values(), tags_word2index.keys()))
tags_index2word[0], tags_index2word[1] = "SOS", "EOS"
    
with open(os.path.join(data_path, 'reports', 'voc.pkl'), 'rb') as f:
    reports_word2index = pickle.load(f)
reports_index2word = dict(zip(reports_word2index.values(), reports_word2index.keys()))
reports_index2word[0], reports_index2word[1] = "SOS", "EOS"

In [17]:
def get_sent_from_tk(tensor_tk, index2word):
    return [index2word[idx.item()] for idx in tensor_tk]

In [18]:
get_sent_from_tk(
    rearrange(tgt, 'b o -> (b o)')
    ,reports_index2word)

['',
 'no',
 'acute',
 'cardiopulmonary',
 'abnormality',
 '.',
 'normal',
 'heart',
 'size',
 'and',
 'mediastinal',
 'contours',
 '.',
 'no',
 'focal',
 'airspace',
 'consolidation',
 '.',
 'no',
 'pleural',
 'effusion',
 'or',
 'pneumothorax',
 '.',
 'chronic',
 'appearing',
 'right',
 'mid',
 'clavicle',
 'injury',
 '.',
 'visualized',
 'bony',
 'structures',
 'otherwise',
 'unremarkable',
 '.',
 '.',
 'SOS',
 '',
 'no',
 'acute',
 'cardiopulmonary',
 'disease',
 '.',
 '.',
 'the',
 'cardiomediastinal',
 'silhouette',
 'is',
 'normal',
 'size',
 'and',
 'configuration',
 '.',
 'pulmonary',
 'vasculature',
 'within',
 'normal',
 'limits',
 '.',
 'the',
 'lungs',
 'are',
 'well',
 'aerated',
 '.',
 'there',
 'is',
 'no',
 'pneumothorax',
 'pleural',
 'effusion',
 'or',
 'focal',
 'consolidation',
 '.',
 '.',
 '',
 'no',
 'acute',
 'cardiopulmonary',
 'disease',
 '.',
 '.',
 'the',
 'cardiomediastinal',
 'silhouette',
 'is',
 'normal',
 'size',
 'and',
 'configuration',
 '.',
 'pulmon

In [71]:
get_sent_from_tk(
    rearrange(src, 'b o -> (b o)')
    ,tags_index2word)

['infiltrates',
 'cardiomegaly',
 'granuloma',
 'granuloma',
 'aorta',
 'left_ventricle',
 'atelectases',
 'atelectasis',
 'granuloma',
 'granuloma',
 'granuloma',
 'granuloma',
 'rib_fracture',
 'rib_fractures',
 'plate_like_atelectasis',
 'pulmonary_atelectasis',
 'atelectases',
 'atelectasis',
 'lymph',
 'lymph_nodes',
 'scarring',
 'scarring',
 'scolioses',
 'scoliosis',
 'tortuous_aorta',
 'aorta',
 'pneumonia',
 'pneumonia',
 'sternotomy',
 'sternotomy',
 'hiatal_hernia',
 'hiatal_hernia',
 'spinal_osteophytosis',
 'thoracic_spondylosis',
 'clip',
 'gallbladder',
 'right_lower_lobe_pneumonia',
 'pneumonia',
 'granuloma',
 'granuloma',
 'aorta',
 'aortic_diseases',
 'apical_granuloma',
 'granuloma',
 'spondylosis',
 'spondylosis',
 'spondylosis',
 'spondylosis',
 'atelectases',
 'atelectasis',
 'tortuous_aorta',
 'aorta',
 'heart_valve_prosthesis',
 'heart_valve_prosthesis_implantation',
 'granulomatous_disease',
 'thoracic_spondylosis',
 'aorta_tortuous',
 'aorta',
 'atherosclero

In [70]:
get_sent_from_tk(
    rearrange(outputs, 'b t v -> (b t) v').max(dim=1)[1].unique()
    ,reports_index2word)

['.']